<a href="https://colab.research.google.com/github/luisfocosta/ai_playground/blob/main/LangChainChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai langchain youtube_transcript_api chromadb unstructured

In [ ]:
import openai
import os

from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import YoutubeLoader

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "<YOUR-KEY>"

In [ ]:
# Option 1: Load documents with DirectoryLoader

from langchain.document_loaders import UnstructuredFileLoader
loader = DirectoryLoader("", glob="transcript.txt")
txt_docs = loader.load_and_split()

# Option 2: Load documents (or transcribe) with YoutubeLoader
loader = YoutubeLoader(video_id="XZY", language="en")
yt_docs = loader.load_and_split()

In [ ]:
# Going with option 1 (txt files)
# Create embeddings
embeddings = OpenAIEmbeddings()
# Write in DB
txt_docsearch = Chroma.from_documents(txt_docs, embeddings)

# Define LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

# Create Retriever
# In case answers are cut-off or you get error messages (token limit)
# use different chain_type
qa_txt = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=txt_docsearch.as_retriever())

In [ ]:
query = "What is the current state of the market and Sam's outlook for 2023?"
qa_txt.run(query)

In [ ]:
# Going with option 2 (yt)

embeddings = OpenAIEmbeddings()
yt_docsearch = Chroma.from_documents(yt_docs, embeddings)

# Define LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

qa_yt = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=yt_docsearch.as_retriever())

In [ ]:
query = "What is the key message of this document?"
qa_yt.run(query)